In [ ]:
# We import numpy and the functions from the python scripts of main_functions_convnets and main_functions_classicNN.
import numpy as np
from main_functions_convnets import *
from main_functions_classicNN import *

In [ ]:
def padding(X, pad=0):
    """ A function which pads a tensor of dimension 4, being (m, height, width, number_channels) the form of the 
    tensor. I want to pad along the axis of height and width. """
    
    X_pad = np.pad(X,((0,0), (pad,pad), (pad,pad), (0,0)), 'constant', constant_values=0)
    
    return X_pad

pad= 1
X = np.ones([3,2,2,3])
X_pad = padding(X)

In [ ]:
def convolution(A_prev_sample, kernel, stride=1, padding='valid'):
    """A function that implements a convolution of a sample of the dataset for one filter."""
    
    #First we calculate what is the height and width of the input. We consider that the input is given by the shape
    # (height, width, depth)
    height_input, width_input, depth = A_prev_sample.shape
    
    #Now we calculate f from the kernel. The kernel (o filter) is given with dimensions (f,f, number of channels)
    f, _, kernel_depth = kernel.shape
    
    # The padding is valid if we don't fill the image with 0's on its borders to manipulate dimensions.

    if padding == 'valid':
        pad = 0
    # If the padding is same it means I am calculating a pad so the input is the same as the output.    
    elif padding == 'same':
        pad = int((f-1)/2)
    
    # Here we calculate the shape of the output and we creatr
    
    height_output, width_output = int((height_input + 2*pad - f)/stride + 1), int((width_input + 2*pad - f)/stride + 1)
    # I create the output of the convolution after having considered f, the stride and the padding.
    conv_output = np.zeros([height_output, width_output])
    
    
    # Now we go in a for loop going through columns and the going down
    
    for height in range(height_output):
        # Now we go through the columns
        for width in range(width_output):
            
            #The current portion of the convolution will be image[height:(height+f), width:(width+f), :]
            current_portion = A_prev_sample[(height*stride):(height*stride+f), (width*stride):(width*stride+f),:]
            
            #Now I do the convolution operation throughout the whole input calling the function from the
            # main_functions_conv script.
            conv_output[height, width] = basic_convolution(current_portion, kernel)
                                         
    return conv_output
    

In [ ]:
# Let's make a function that makes the convolution operation in a layer on all the filters and all the samples.
def convolution_forward(A_prev, W, stride=1, padding='valid'):
    """A function that takes the m samples of A_prev, all the filters from W and does an iteration in one
    layer of a convolution network."""
    
    # First we calculate the dimensions of A_prev --> (m, n_H_prev, n_W_prev, n_c_prev)
    m, n_H_prev, n_W_prev, n_C_prev_A_prev = A_prev.shape
    
    # Now we calculate the dimensions of the filters W --> (f, f, n_c_prev, n_c)
    f, _, n_C_prev_W, n_C = W.shape
    
    #If the padding is valid, then the pad is equals to 0.
    if padding == 'valid':
        pad = 0
    
    # Now we assert that the number of channels of A n_c_prev and of filters are the same.
    assert (n_C_prev_A_prev == n_C_prev_W), "The number of channels of the previous layer must be the same\
    in the kernels as well as in the activation A_prev."
    
    # Now we calculate the height and width of the output.
    
    n_H, n_W = int((n_H_prev + 2*pad - f)/stride + 1),\
        int((n_W_prev + 2*pad - f)/stride + 1)
     
    # We now have the dimensions of the output. I will call it Z.
    Z = np.zeros([m, n_H, n_W, n_C])
    
    # Now we go through the m samples and the number of channels n_C in the layer:
    for i in range(m):
        # We go to the i sample of the dataset
        A_prev_sample = A_prev[i]
        
        for c in range(n_C):
            # Now let's get each of the filters of the network and apply the convolution. We have tp slice the
            # last dimension so we get the kernel of dimensions (f,f, n_C). This is W[:,:,:,c]
            kernel = W[:,:,:,c]
            # Once we get the kernel of channel c we can do the convolution to the sample and we can insert it
            # in the i'th sample of the network in the channel c.
            Z[i,:,:,c] = convolution(A_prev_sample, kernel, stride, padding)
            
    #Once convolution is made through the m samples and the n_C channels we have obtained the convolution in that 
    # layer and we can return Z.
    
    return Z
    

In [ ]:
# Now we create a function that adds the bias to the output we got from the convolution making broascast with numpy.
# This allows us to add to the 4D array of dimensions (number_samples, n_height, n_width, number of channels)
# a 2D array of dimensions (1, number of channels) to add the bias along the axis of the channels.

def add_bias(A, biases):
    """A function that adds the bias to each of the outputs we get from each filter applied."""
    # First, we make sure that the number of biases applied to the output A is the same as the number of channels
    # (The last dimension of A). We consider that we are getting as input a 4D array of shape (m, n_H, n_W, n_C).
    # We first assert that.
    n_dim_A = len(A.shape)
    n_dim_b = len(biases.shape)
    assert(n_dim_A == 4 and n_dim_b == 2), "The number of dimensions of A must be 4 and the number of dimensions \
    of b must be 2."
    
    _, _ , _, n_C_A = A.shape
    _, n_C_b = biases.shape
    
    assert(n_C_A == n_C_b), "The number of biases must be the same as the number of channels of A."
    
    A_with_bias = A + biases
    
    return A_with_bias
    

In [ ]:
def activation_convolution(A_prev, W, biases, activation_name='relu', stride=1, padding='valid'):
    """A function that makes all the operations involved in a convolutional layer. This is applying all the
    convolution filters given by the 4D array W, adding the bias to the 4D array obtained, and finally
    applying an activation function. The dimensions given must be the following:
    A_prev ---> (number_samples, height, width, number_channels_prev)
    W ---> (f, f, number_channels_prev, number_filters) being f the height and width of the filters 
    applied (Considering that width=height) 
    biases ---> (1, number_filters)"""
    
    # We first make sure that the dimensions given are appropiate: 
    # A_prev is a 4D array of shape :(m, n_H_prev, n_W_prev, n_channels_prev)
    # W is a 4D array with all the filters in that layer and shape : (f, f, n_channels_prev, n_channels)
    # biases is a 2D array of shape : (1, n_channels)
    assert (len(A_prev.shape) == 4 and len(W.shape) == 4 and len(biases.shape) == 2), "The dimensions must be :\
    A_prev a 4D array with shape (number_samples, height, width, number_channels_prev) ; W a 4D array with shape \
    (f, f, number_channels_prev, number_filters) and b a 2D array with shape (1, number_filters)"
    
    # First we apply the convolution with the function convolution_forward
    conv_output = convolution_forward(A_prev, W, stride, padding)
    # Next we add the bias the the convolution output
    Z = add_bias(conv_output, biases)
    
    # Finally we apply the activation function with the function activation_functions.
    A = activation_functions(Z, activation_name)
    
    # Finally we return the activation obtained.
    return A

In [ ]:
# Test box
np.random.seed(1)
n_H = 10
n_W = 10
n_C_prev = 4
n_C = 5
f = 5
m = 3
A_prev = np.random.randint(low=0, high=2, size=(m, n_H, n_W, n_C_prev))
W = np.ones([f, f, n_C_prev, n_C])
biases = np.random.randint(low=0, high=4, size=(1,n_C))
print('A_prev with number of samples={}, height={}, width={}, and number of channels={}'.format(m, n_H, n_W, n_C_prev), '\n')
print('W with f={}, n_channels_prev={} and number of filters={} is:\n {}'.format(f, n_C_prev, n_C, W))
print('biases={}'.format(biases))

A = activation_convolution(A_prev, W, biases, activation_name='relu', stride=1, padding='valid')
print('A obtained with shape ={} and result:\n{}'.format(A.shape, A))

In [ ]:
# Starting the operations of max pooling and average pooling on one channel.
def pooling_sample(A_prev_sample, f, pooling_type='max', stride=1):
    """A function  that makes the pooling operation for one sample. By default it makes the max pooling operation."""
    #As we have one sample, we have a 3D array with shape (n_H_prev, n_W_prev, n_C_prev). We can assert this
    # to make sure we give the right arguments.
    assert (len(A_prev_sample.shape) == 3), "The A_prev_sample must be a 3D array"
    
    n_H_prev, n_W_prev, n_C_prev = A_prev_sample.shape
    
    # Now we get the dimensions obtained with the f and stride:
    n_H = get_n_H(n_H_prev, f, stride)
    n_W = get_n_W(n_W_prev, f, stride)

    # Now we can create the output of the sample with dimensions height, width and the number of channels are the
    # same as in the input given.
    
    Z_sample = np.zeros([n_H, n_W, n_C_prev])
    
    # And now we can operate through the sample
    for c in range(n_C_prev):
        # We get the slice of A_prev in the channel c, so we are getting the matrix of channel c (2D-array)
        A_sample_channel = A_prev_sample[:,:,c]
        
        # Now we have the 2D array of channel c we can iterate over the height and the width of the matrix.
        for h in range(n_H):
            for w in range(n_W):
                # Now we need to iterate through the matrix in the channel c considering the stride and f:
                current_portion = A_sample_channel[(h*stride):(h*stride+f),\
                                                   (w*stride):(w*stride+f)]
                
                # Now we can extract the max or the mean of that portion depending if we want to do a max-pooling
                # or average pooling.
                if pooling_type.lower() == 'max':
                    Z_sample[h, w, c] = np.max(current_portion)
                elif pooling_type.lower() == 'average':
                    Z_sample[h, w, c] = np.mean(current_portion)
    
    # Once done in all the channels, we can return Z_sample
    return Z_sample
                
    

In [ ]:
# Now we need to develop the function that makes the pooling through all the samples. In this case we receive
# as input a 4D array with dimensions (m, n_H_prev, n_W_prev, n_C_prev). We make as default stride=1 and the 
# pooling type max_pooling. In the pooling layers it is never used the padding because the main goal of this 
# operation is to reduce the dimensions of the input.

def pooling_forward(A_prev, f, pooling_type='max', stride=1):
    """A function that makes the pooling operation for all the samples of that layer"""
    
    # Let's assert the dimensions of A_prev as a 4D-array with shape (m, n_H_prev, n_W_prev, n_C_prev)
    assert (len(A_prev.shape) == 4), "A_prev must be a 4D-array with shape (m, n_H_prev, n_W_prev, n_C_prev)"
    
    # Now we get the dimensions of A_prev.
    m, n_H_prev, n_W_prev, n_C_prev = A_prev.shape
    
    #Let's get the dimensions of n_H and n_W considering f and the stride.
    n_H = get_n_H(n_H_prev, f, stride)
    n_W = get_n_W(n_W_prev, f, stride)
    
    # Now we can create the output
    Z = np.zeros([m, n_H, n_W, n_C_prev])
    
    
    # After having the dimensions, we can go in a for loop for the m samples and use the last function 
    # pooling_sample that makes the pooling for one sample.
    
    for i in range(m):
        
        # We consider the sample i from A_prev
        A_prev_sample = A_prev[i]
        #Once we have the sample i we can use the function pooling_sample
        Z[i] = pooling_sample(A_prev_sample, f, pooling_type, stride)
    return Z    

In [ ]:
# As we are using the notes of https://www.coursera.org/learn/convolutional-neural-networks we are going to 
# consider that the pooling is made inside a convolutional layer.
def conv_and_pool(A_prev, W, biases, activation_name, stride_conv, padding_conv, f_pool,
                  stride_pool, pooling=False, pool_type='max'):
    """A function that takes the input A_prev and makes a convolution with its respective parameters and pooling
    if that layer has one."""
    A_convolved = activation_convolution(A_prev, W, biases, activation_name, stride_conv, padding_conv)
    
    if pooling is False:
        print('No pooling in this layer')
    else:
        print('Pooling  in this layer with type {} pooling'.format(pool_type))
        A_convolved_and_pooled = pooling_forward(A_convolved, f_pool, pool_type, stride_pool)
    
    # Once we have made the convolution and pooling we can return the result of this layer
    
    return A_convolved_and_pooled

In [ ]:
# A function that flattens the first fully connected layer. It makes it by reshaping it into a matrix of shape
# (m, number_features), being the number of features the product of n_H, n_W and n_C
def fully_connected(A):
    """Reshaping the 4D array into a 2D array with shape (m, n_H*n_W*n_C)"""
    m = A.shape[0]
    features = np.prod(A.shape[1:])
    print('number of samples={} and number of features={}'.format(m,features))
    A_FC = A.reshape(m,-1)
    
    return A_FC

In [ ]:
def forward_propagation_convnet(X, filters_dict, biases_dict, network_structure):
    pass

In [ ]:
network_structure = {'layer_1':{'conv':{'f':5, 'stride':2, 'padding':0}, 'pool':{'type':'max','f':2, 'stride':2}},\
                'layer_2':{'conv': {'f':4, 'stride':2, 'padding':0}, 'pool':{'type':'max', 'f':2, 'stride':2}},\
                    'layer_3':{'FC':{'activation_name':'relu'}},\
                     'layer_4':{'FC':{'number_neurons':120, 'activation_name':'relu'}},\
                    'layer_5':{'FC':{'number_neurons':84, 'activation_name':'relu'}},\
                     'layer_6':{'FC':{'number_neurons':10, 'activation_name':'softmax'}}}

               
network_structure['layer_2']



In [ ]:
np.random.seed(1)

m = 3
n_H_prev = 3
n_W_prev = 2
n_C_prev = 4

M = np.random.randint(low=0, high=5, size=(m, n_H_prev, n_W_prev, n_C_prev))

print('M with shape {} is:\n{}'.format(M.shape, M))
print('M flattened =\n{}'.format(M.flatten()))
print('M reshaped = \n{}'.format(M.reshape((m, n_H_prev*n_W_prev*n_C_prev))))
print('Another form of reshape is=\n{}'.format(fully_connected(M)))

In [ ]:
# Now we are going to load some data of cats and dogs to try our algorithm.
# I first import the libraries I need to interact with the directories.
import os
import matplotlib.pyplot as plt
import matplotlib.image as img
import cv2

current_dir = '/home/inapifer/Desktop/Deep_Learning'
i = 1000

rute_to_sample_cat = 'Data_cats_dogs/training_set/cats/cat.' + str(i) + '.jpg'


abs_rute_cat = os.path.join(current_dir, rute_to_sample_cat)

cat_matrix = img.imread(abs_rute_cat)
